In [1]:
import copy
from dataclasses import dataclass
from enum import Enum
import logging
import numpy as np
from typing import List, Optional
from __future__ import annotations

# Background
The purpose of this Jupyter notebook is to simulate blackjack

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Run Parameters

In [3]:
class HandAction(Enum):
    STAND = 1
    HIT = 2
    DOUBLE = 3
    SPLIT = 4
    
    
class HandStatus(Enum):
    BUST = 0
    STAND = 1
    HARD = 2
    SOFT = 3
    DOUBLE = 4
    SPLIT = 5

    
@dataclass
class Card:
    name: str
    value: int
    hidden: bool = None
    is_soft: bool = None
        
    def __repr__(self):
        return self.name
        
        
UNIQUE_CARDS = [
    Card('Two', 2),
    Card('Three', 3),
    Card('Four', 4),
    Card('Five', 5),
    Card('Six', 6),
    Card('Seven', 7),
    Card('Eight', 8),
    Card('Nine', 9),
    Card('Ten', 10),
    Card('Jack', 10),
    Card('Queen', 10),
    Card('King', 10),
    Card('Ace', 11)
]

@dataclass
class Hand:
    cards: List[Optional[Card, Hand]] = None
    bet: float = None
    hand_status: HandStatus = HandStatus.HARD
    is_dealer: bool = None
    is_double: bool = None
        
    def __post_init__(self):
        if self.cards is None:
            self.cards = []
        
        if self.cards:
            self.calculate_value()
        
    @property
    def value(self):
        if isinstance(self.cards[0], Card):
            return self.calculate_value()
        
    @property
    def is_blackjack(self):
        not_split = all([isinstance(card, Card) for card in self.cards])
        return (not_split and len(self.cards) == 2 and
                    ((self.cards[0].name == 'Ace' and self.cards[1].value == 10) or 
                     (self.cards[1].name == 'Ace' and self.cards[0].value == 10)))
                
    def calculate_value(self):
        aces = []
        value = 0
        value += sum([card.value for card in self.cards if card.name != 'Ace' and not card.hidden])
        
        # handle the aces
        is_soft = False
        for ace in [card for card in self.cards if card.name == 'Ace' and not card.hidden]:
            if value + 11 > 21:
                value += 1
            else:
                value += 11
                is_soft = True
                
        if is_soft:
            self.hand_status = HandStatus.SOFT
            
        if value > 21:
            self.hand_status = HandStatus.BUST
            
        return value
    
    def play(self,
             table: Table,
             money: float):
        continue_play = True
        while continue_play and self.hand_status != HandStatus.BUST:
            action = play_hand(self, table.dealer.hand, money, self.bet)
            continue_play, table, money = self.resolve_play(action, table, money)
        
        return table, money
    
    def dealer_play(self,
                    table):
        for card in self.cards:
            card.hidden = False
            
        while (self.hand_status != HandStatus.BUST or (self.hand_status == HandStatus.SOFT and self.value <= 17) \
                   or (self.hand_status == HandStatus.HARD and self.value < 17)):
            __, table, __ = self.resolve_play(HandAction.HIT, table)
        return table
            
    def resolve_play(self,
                     action: HandAction,
                     table: Table,
                     money: float = None):
        if action == HandAction.STAND:
            self.hand_status = HandStatus.STAND
            
            return False, table, money
        elif action == HandAction.HIT:
            table.deal_one_card(self)
            
            return True, table, money
        elif action == HandAction.DOUBLE:
            money -= self.bet
            self.hand_status = HandStatus.DOUBLE
            table.deal_one_card(self)
            
            return False, table, money
        elif action == HandAction.SPLIT:
            self.cards = [Hand(cards=[self.cards.pop()], bet=self.bet),
                          Hand(cards=[self.cards.pop()], bet=self.bet)]
            money -= self.bet
            self.bet = 0
            self.hand_status = HandStatus.SPLIT
            for split_hand in self.cards:
                table.deal_one_card(split_hand)
                table, money = split_hand.play(table,
                                               money)
            return False, table, money
        else:
            raise ValueError(f"HandAction {action} not recognized!")
            
    def resolve_bet(self,
                    dealer_hand):
        result = 0
        if self.hand_status == HandStatus.SPLIT:
            for split_hand in self.cards:
                result += split_hand.resolve_bet(dealer_hand)
                
                print(f"result: {result} / {split_hand.cards} / {dealer_hand.cards}")
        elif self.hand_status != HandStatus.BUST and ~dealer_hand.is_blackjack:
            if self.value == dealer_hand.value:
                result += self.bet
            elif dealer_hand.hand_status == HandStatus.BUST or self.value > dealer_hand.value:
                result += self.bet*2
            print(f"result: {result} / {self.cards} / {dealer_hand.cards}")
        
        return result
        

def more_if_possible(more_action, other_action, money, bet):
    if money >= bet or money == -1:
        return more_action
    else:
        return other_action
        
        
def play_hand(own: Hand,
              dealer: Hand,
              money: float,
              bet: float):
    if len(own.cards) == 2 and own.cards[0].name == own.cards[1].name:
        if own.cards[0].value <= 3:
            if dealer.value <= 7:
                return more_if_possible(HandAction.SPLIT, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.cards[0].value == 4:
            if dealer.value == 5 and dealer.value == 6:
                return more_if_possible(HandAction.SPLIT, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.cards[0].value == 5:
            if dealer.value <= 9:
                return more_if_possible(HandAction.DOUBLE, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.cards[0].value == 6:
            if dealer.value <= 6:
                return more_if_possible(HandAction.SPLIT, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.cards[0].value == 7:
            if dealer.value <= 7:
                return more_if_possible(HandAction.SPLIT, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.cards[0].value == 8 or own.cards[0].value == 11:
            return more_if_possible(HandAction.SPLIT, HandAction.HIT, money, bet)
        elif own.cards[0].value == 9:
            if dealer.value == 7 or dealer.value >= 10:
                return HandAction.STAND
            else:
                return more_if_possible(HandAction.SPLIT, HandAction.STAND, money, bet)
        else:
            return HandAction.STAND
    elif own.hand_status == HandStatus.HARD:
        if own.value <= 8:
            return HandAction.HIT
        elif own.value == 9:
            if dealer.value >= 3 and dealer.value <= 6:
                return more_if_possible(HandAction.DOUBLE, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.value == 10:
            if dealer.value <= 9:
                return more_if_possible(HandAction.DOUBLE, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.value == 11:
            if dealer.value <= 10:
                return more_if_possible(HandAction.DOUBLE, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.value == 12:
            if dealer.value >= 4 and dealer.value <= 6:
                return HandAction.STAND
            else:
                return HandAction.HIT
        elif own.value <= 16:
            if dealer.value <= 6:
                return HandAction.STAND
            else:
                return HandAction.HIT
        else:
            return HandAction.STAND
    elif own.hand_status == HandStatus.SOFT:
        if own.value <= 14:
            if dealer.value >= 5 and dealer.value <= 6:
                return more_if_possible(HandAction.DOUBLE, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.value <= 16:
            if dealer.value >= 4 and dealer.value <= 6:
                return more_if_possible(HandAction.DOUBLE, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.value == 17:
            if dealer.value >= 3 and dealer.value <= 6:
                return more_if_possible(HandAction.DOUBLE, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        elif own.value == 18:
            if dealer.value == 2 or dealer.value == 7 or dealer.value == 8:
                return HandAction.STAND
            elif dealer.value >= 3 and dealer.value <= 6:
                return more_if_possible(HandAction.DOUBLE, HandAction.HIT, money, bet)
            else:
                return HandAction.HIT
        else:
            return HandAction.STAND
    else:
        raise ValueError(f"Hand status {own.hand_status} not detected!")

1. Create deck
2. Shuffle deck
3. Cut deck
4. Retrieve bets
5. deal cards
6. Optional (insurance)
7. Play player hands
8. Play dealer hands
9. Pay out
10. Clean up cards

In [4]:
def bound(low, high, value):
    return max(low, min(high, value))

class Shoe:
    def __init__(self,
                 cards: List[Card],
                 cut_loc: int = None):
        self.cards = cards
        self.cut_loc = cut_loc
        
    @classmethod
    def create_shoe(cls,
                    num_decks: int = 6,
                    num_suites: int = 4,
                    shuffle: bool = True,
                    cut: bool = True,
                    unique_cards: List[Card] = UNIQUE_CARDS,
                    seed = None):
        np.random.seed(seed)
        cards = []
        for _ in range(num_decks):
            for _ in range(num_suites):
                cards.extend([copy.deepcopy(card) for card in unique_cards]) 
        
        if shuffle:
            np.random.shuffle(cards)
            
        if cut:
            cut_perc = bound(0, 0.5, np.random.normal(0.25, 0.05))
            cut_loc = round(cut_perc*len(cards))
            cards = cards[-cut_loc:] + cards[:-cut_loc]
        
        return cls(cards, cut_loc)
    
    def __len__(self):
        return len(self.cards)

In [5]:
@dataclass
class Seat:
    money: int
    track_money: bool = True
    is_dealer: bool = False
    hand: Hand = None
    in_play: bool = True
        
    @classmethod
    def create_pc(cls, money):
        return cls(money, track_money=True, is_dealer=False)
    
    @classmethod
    def create_npc(cls):
        return cls(money=-1, track_money=False, is_dealer=False)
    
    @classmethod
    def create_dealer(cls):
        return cls(money=-1, track_money=False, is_dealer=True)
    
    def make_min_bet(self,
                     min_bet: int):
        if not self.track_money or self.money >= min_bet:
            if self.track_money:
                self.money -= min_bet
            self.hand = Hand(bet=min_bet)
            self.in_play = True
            return
        else:
            self.in_play = False
            
    def play_hand(self,
                  table: Table):
        table, money = self.hand.play(table,
                                      self.money)
        self.money = money
        return table

In [6]:
@dataclass
class Table:
    seats: List[Seat] = None
    dealer: Seat = None
    shoe: Shoe = None
    min_bet: int = 20
        
    @classmethod
    def create_table(cls, seat_money_vals: list, shoe_kwargs: dict = None):
        if shoe_kwargs is None:
            shoe_kwargs = {}
            
        seats = []
        for seat_money in seat_money_vals:
            if seat_money == -1:
                seats.append(Seat.create_npc())
            else:
                seats.append(Seat.create_pc(seat_money))
        return cls(seats=seats,
                   dealer=Seat.create_dealer(),
                   shoe=Shoe.create_shoe(**shoe_kwargs))
            
    def deal_one_card(self, hand: Hand, make_hidden: bool = False):
        card_to_deal = self.shoe.cards.pop()
        if make_hidden:
            card_to_deal.hidden = True
        
        hand.cards.append(card_to_deal)
        hand.calculate_value()
        
        return hand.hand_status != HandStatus.BUST
    
    def deal_one_card_to_all(self):
        for seat in self.seats:
            if seat.in_play:
                self.deal_one_card(seat.hand)
                
    def gather_bets(self):
        for seat in self.seats:
            seat.make_min_bet(self.min_bet)
        
        self.dealer.hand = Hand(is_dealer=True)
        
    def deal(self):
        if len(self.shoe) > self.shoe.cut_loc:
            # first card
            self.deal_one_card_to_all()
            self.deal_one_card(self.dealer.hand, make_hidden=False)
            
            # second card
            self.deal_one_card_to_all()
            self.deal_one_card(self.dealer.hand, make_hidden=True)
                
            return True
        else:
            return False
        
    def resolve_insurance(self):
        pass
        
    def play_all_hands(self):
        for seat in self.seats:
            if seat.in_play:
                self = seat.play_hand(self)
        self = table.dealer.hand.dealer_play(self)
    
    def resolve_bets(self):
        for seat in self.seats:
            result = seat.hand.resolve_bet(self.dealer.hand)
            seat.money += result
            
    def clean_table(self):
        for seat in self.seats:
            seat.hand = None
        self.dealer.hand = None

In [7]:
table = Table.create_table([1000, 1000, 1000, 1000, 1000], shoe_kwargs={'num_decks': 10, 'seed': 39303})

In [8]:
table.shoe.cut_loc

83

In [9]:
for i, seat in enumerate(table.seats):
    print(f"Seat: {i} / Money: {seat.money: >3d}")
print(f"Seat: D / Money: {table.dealer.money: >3d}")

Seat: 0 / Money: 1000
Seat: 1 / Money: 1000
Seat: 2 / Money: 1000
Seat: 3 / Money: 1000
Seat: 4 / Money: 1000
Seat: D / Money:  -1


In [10]:
table.gather_bets()

In [11]:
for i, seat in enumerate(table.seats):
    print(f"Seat: {i} / Money: {seat.money: >3d} / Bet: {seat.hand.bet: >4d} / Hand: {str(seat.hand.cards): >14}")
print(f"Seat: D / Money: {table.dealer.money: >3d} / Bet: {str(table.dealer.hand.bet): >2} / Hand: {str(table.dealer.hand.cards): >14}")

Seat: 0 / Money: 980 / Bet:   20 / Hand:             []
Seat: 1 / Money: 980 / Bet:   20 / Hand:             []
Seat: 2 / Money: 980 / Bet:   20 / Hand:             []
Seat: 3 / Money: 980 / Bet:   20 / Hand:             []
Seat: 4 / Money: 980 / Bet:   20 / Hand:             []
Seat: D / Money:  -1 / Bet: None / Hand:             []


In [12]:
table.deal()

True

In [13]:
for i, seat in enumerate(table.seats):
    print(f"Seat: {i} / Money: {seat.money: >3d} / Bet: {seat.hand.bet: >4d} / Hand: {str(seat.hand.cards): >14}")
print(f"Seat: D / Money: {table.dealer.money: >3d} / Bet: {str(table.dealer.hand.bet): >2} / Hand: {str(table.dealer.hand.cards): >14}")

Seat: 0 / Money: 980 / Bet:   20 / Hand:     [Ten, Two]
Seat: 1 / Money: 980 / Bet:   20 / Hand:    [Two, King]
Seat: 2 / Money: 980 / Bet:   20 / Hand:   [Six, Eight]
Seat: 3 / Money: 980 / Bet:   20 / Hand:   [Queen, Two]
Seat: 4 / Money: 980 / Bet:   20 / Hand:     [Ace, Ace]
Seat: D / Money:  -1 / Bet: None / Hand:  [Seven, Nine]


In [14]:
table.play_all_hands()

In [15]:
for i, seat in enumerate(table.seats):
    print(f"Seat: {i} / Money: {seat.money: >3d} / Bet: {seat.hand.bet: >4d} / Hand: {str(seat.hand.hand_status): >14} / Hand: {str(seat.hand.cards): >14}")
print(f"Seat: D / Money: {table.dealer.money: >3d} / Bet: {str(table.dealer.hand.bet): >2} / Hand: {str(table.dealer.hand.cards): >14}")

Seat: 0 / Money: 980 / Bet:   20 / Hand: HandStatus.BUST / Hand: [Ten, Two, Queen]
Seat: 1 / Money: 980 / Bet:   20 / Hand: HandStatus.BUST / Hand: [Two, King, Four, Nine]
Seat: 2 / Money: 980 / Bet:   20 / Hand: HandStatus.BUST / Hand: [Six, Eight, Two, Eight]
Seat: 3 / Money: 980 / Bet:   20 / Hand: HandStatus.BUST / Hand: [Queen, Two, Queen]
Seat: 4 / Money: 960 / Bet:    0 / Hand: HandStatus.SPLIT / Hand: [Hand(cards=[Ace, Four, Queen, Four], bet=20, hand_status=<HandStatus.STAND: 1>, is_dealer=None, is_double=None), Hand(cards=[Ace, Seven], bet=20, hand_status=<HandStatus.STAND: 1>, is_dealer=None, is_double=None)]
Seat: D / Money:  -1 / Bet: None / Hand: [Seven, Nine, King]


In [16]:
table.resolve_bets()

result: 40 / [Ace, Four, Queen, Four] / [Seven, Nine, King]
result: 40 / [Ace, Four, Queen, Four] / [Seven, Nine, King]
result: 40 / [Ace, Seven] / [Seven, Nine, King]
result: 80 / [Ace, Seven] / [Seven, Nine, King]


In [17]:
for i, seat in enumerate(table.seats):
    print(f"Seat: {i} / Money: {seat.money: >3d} / Bet: {seat.hand.bet: >4d} / Hand: {str(seat.hand.cards): >14}")
print(f"Seat: D / Money: {table.dealer.money: >3d} / Bet: {str(table.dealer.hand.bet): >2} / Hand: {str(table.dealer.hand.cards): >14}")

Seat: 0 / Money: 980 / Bet:   20 / Hand: [Ten, Two, Queen]
Seat: 1 / Money: 980 / Bet:   20 / Hand: [Two, King, Four, Nine]
Seat: 2 / Money: 980 / Bet:   20 / Hand: [Six, Eight, Two, Eight]
Seat: 3 / Money: 980 / Bet:   20 / Hand: [Queen, Two, Queen]
Seat: 4 / Money: 1040 / Bet:    0 / Hand: [Hand(cards=[Ace, Four, Queen, Four], bet=20, hand_status=<HandStatus.STAND: 1>, is_dealer=None, is_double=None), Hand(cards=[Ace, Seven], bet=20, hand_status=<HandStatus.SOFT: 3>, is_dealer=None, is_double=None)]
Seat: D / Money:  -1 / Bet: None / Hand: [Seven, Nine, King]


In [18]:
table.clean_table()

In [19]:
for i, seat in enumerate(table.seats):
    print(f"Seat: {i} / Money: {seat.money: >3d} / Hand: {str(seat.hand): >5}")
print(f"Seat: D / Money: {table.dealer.money: >3d} / Hand: {str(table.dealer.hand): >5}")

Seat: 0 / Money: 980 / Hand:  None
Seat: 1 / Money: 980 / Hand:  None
Seat: 2 / Money: 980 / Hand:  None
Seat: 3 / Money: 980 / Hand:  None
Seat: 4 / Money: 1040 / Hand:  None
Seat: D / Money:  -1 / Hand:  None


Double Down
1. Get cards
2. Make Decision --> stand
3. Pay Money
4. Set Double down status
5. Get another card
6. Check bust status

Hit
1. Get cards
2. Make Decision --> Hit
3. Get card
4. Check bust status
5. Make Decision...

Stand
1. Get cards
2. Make decision --> Stand

Split
1. Get cards
2. Make decision --> split
3. Pay Money
4. Set Split status
5. deal cards
6. make decision on each